# Import Libraries

In [ ]:
using Plots 

Plots.plotly()
Plots.default(size=(800,300)); # Set default plot canvas size
Plots.default(label=""); # Turn of legends by default
Plots.default(ticks=:native);

# Energy via Integration

The energy $E$ of a signal $x(t)$ is defined by the integration

$E = \int_{-\infty}^{\infty}| x(t) |^2 dt$

which is approximated by the sum

$E = \sum_{n=0}^{N -1 } p(t_1 + \Delta t) \Delta t$

The sum becomes more accurate with an increase in the time step $\Delta t$. The code below generates the energy via numerical integration.

In [ ]:
t1= -1; t2= 1; Δt= 0.001;
t=t1:Δt:t2;
 # or use t=range(t1, t2, step=Δt)

f = 1       # T = 1 s <=> f = 1 Hz
f0 = 6      
ω0 = 2*pi*f0    # fundamental frequency w0
A = 10

rect(t) = (abs.(t) .<= 0.5) .* 1.0
myfunc(t) = A*rect(t).*cos.(ω0*t)
 
x = myfunc.(t)

# Function to calculate the instantaneous power
InstPower(x) = (abs(x))^2

Energy(x,Δt) = sum( InstPower.(x) )*Δt
#  Energy is calculated by integrating power

fig_x0 = plot(t,x); 
xlabel!("t (s)")
ylabel!("x(t)")
title!("Sinusoidal pulse x(t) obtained by multiplying sinusoid by rectangular pulse function")
display(fig_x0);


fig_x1 = plot(t, InstPower.(x));
xlabel!("t (s)")
ylabel!("p(t)")
title!("Instantaneous power p(t) of sinusoidal pulse x(t)")

display(fig_x1)
println("Energy = $(Energy(x,Δt)) J" );


# Energy of impulse response of an ideal LPF 

We define the impulse response $h(t)$ of an ideal lowpass filter (LPF) with bandwidth $B$ Hz
as 

$h(t) = 2Sa(2 \pi t)$

We use the above method to determine the energy of the impulse response by approximating it by the sum

$E = \sum_{n=0}^{N -1 } p(t_1 + \Delta t) \Delta t$

For more accurate results, we choose a wide time interval $[t_1, t_2]$ and a small time step $\Delta t$.

In [ ]:
t1= 0.0000000000001; t2= 100; Δt= 0.001;
t=t1:Δt:t2;
B_a = 1     # bandwidth B = 1 Hz

Sa(t) = sin.(t)./t

# rect(t) = (abs.(t-1) .<= 0.5) .* 1.0
# myimpulse(t) = 2*B_a*rect(t).*Sa(2*pi*B_a*(t-1))
myimpulse(t) = 2*B_a*Sa(2*pi*B_a*(t))

h = myimpulse.(t)

InstPower(h) = (abs(h))^2
Energy(h,Δt) = sum( InstPower.(h) )*Δt

fig_h = plot(t,h);
xlabel!("t (s)")
ylabel!("h(t)")
title!("Impulse response h(t) of an ideal LPF with bandwidth B = 1 Hz")
display(fig_h);

println("Energy = $(2*Energy(h,Δt)) J" );

In the above, the time interval is started at $t_1 \approx 0$ and not exactly at $t = 0$ so that the energy converges. Since the function is symmetric about the vertical axis, we perform the calculation on the right half of the impulse response then double the result to obtain the required energy.

Plotting filter impulse response

The impulse response $h(t)$ of an ideal LPF of bandwidth $B$ = 1 Hz and unity gain is the $\sin(x)$ over $x$ function:
$h(t) = 2 Sa(2\pi t)$, where $Sa(x) = sin(x)/x$. Below is the code for creating a sampled sinusoid.

In [ ]:
Δt = 0.001          # time step
t = -2*pi:Δt:2*pi;        # steps of 0.01s from 0s to 1s
j = im;

N = length(t)
println("Number of samples = $(N).")

A = 2
B = 1   # Frequency in Hz
x = 2*pi*B*t;

h_sine = A*sin.(x); # Array holding values of the numerator of the Sa-function
h_denom = x         # The denominator of the Sa-function
h_lpf = h_sine./h_denom; # The impulse response as a Sa-function


# Plot the sine waveform in the numerator of the Sa-function

Showing the plot of the numerator of the Sa-function representing the impulse function as a sine waveform with amplitude 2. 

In [ ]:
fig_sine = plot(t, h_sine)
xlabel!("t (s)");
ylabel!("h_sine")
title!("Plot of sine wave vs time")
display(fig_sine)

# Plot of the denominator of the Sa-function

In [ ]:
fig_denom = plot(t, h_denom)
xlabel!("t (s)");
ylabel!("h_denom")
title!("Plot of 2 pi t vs t")
display(fig_denom)

# Plot of the impulse response function in the time domain

The impulse function of a LPF is given by $h_{sin}$ divided by $h_{denom}$.

In [ ]:
fig_impulse_response = plot(t, h_lpf)
xlabel!("t (s)")
ylabel!("h(t)")
title!("Plot of impulse response vs t")
display(fig_impulse_response)

# Approximating the width of the main lobe of impulse response

The width of the main lobe is measured between the 3dB points, i.e. the distance between the points on the above plot where the value of the impulse response function on the y-axis is 70.7% of the maximum height of 2. These points have a y-value of 1.414. This corresponds to $x_1$ = -0.221 and $x_{2}$ = 0.221. The distance between two points is 

d = $\sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}$

Therefore, the width is 0.221 - (-0.221) = 0.44 = 0.44/1 = 0.44/B.

Alternatively, the value of $t$ where at the 3dB points corresponds to the intersection of the curve of $y_1 = 2 sin(2\pi t)$ and $y_2 = 2\pi t / \sqrt(2)$.

In [ ]:
# Δt = 0.001          # time step
# t = -2*pi:Δt:2*pi;  

plot(t, h_sine)
fig_intersect = plot!(t, sqrt(2).*h_denom)

xlabel!("t (s)")
ylabel!("y(t)")
title!("Plot showing the intersection of sinx and x/sqrt(2)")

display(fig_intersect)


From the above plot, the intersection is at $t = 0.221$. We can verify this by evaluating the impulse response $h(t)$ at this value of $t$, i.e. 

$h(\pm 0.221) = 2Sa (2\pi(\pm 0.221)) \approx 1.414$

This shows that the width of the main lobe at the -3dB point is 0.44 and corresponds to the points where the value of the impulse response function is 1.414, or 70.7% of the amplitude 2. 

# The impulse response of an ideal BPF

We can use the same technique as above to plot the impulse response of the bandpass filter (BPF) by noting that the result is a product of the Sa-function and a cosine function.

In [ ]:
f0 = 4                              # fundamental frequency in Hz
w0 = 2*pi*f0                        # fundamental frequency in rad/s

h_bpf = (2*B*sin.(pi*B*t)./(pi*B*t)).*(cos.(w0*t));

fig_bpf = plot(t, h_bpf)
xlabel!("t (s)");
ylabel!("Impulse response of BPF")
title!("Plot of the h_bpf vs time")

display(fig_bpf)


For further analysis, we can plot the factors in the expression describing the impulse response on the same axis to see the corresponding values that are being multiplied together.

In [ ]:
h_sa_factor = 2*B*sin.(pi*B*t)./(pi*B*t);           # the factor of the Sa-function of the impulse resp
h_cos_factor = cos.(w0*t);                          # the cosine factor of the impulse resp

fig_sa_factor = plot(t, h_sa_factor)
fig_final = plot!(t, h_cos_factor)
xlabel!("t (s)")
ylabel!("y(t)")
title!("Showing the functions that were multiplied together to obtain h_bpf")
display(fig_final)


We see that when multiplying values within the main lobe of the Sa-function with the values of the sinusoid, we obtain the central part of the impulse response of the BPF whose value at the maximum corresponds to the height of the Sa-function.

# Step response via integration

The response of a LPF can be found by integrating the impulse response $h_{lpf}(t)$:

$y(t) = \int_{-\infty}^{t} {h_{lpf}(\tau)} d\tau$

We can perform this operation for a range of $t$ values by using the *cumsum(X)* function which cummulatively sums the values in the array X, creating an output array.

In [ ]:
y_lpf = cumsum(h_lpf)*Δt                        # cummulatively sums values of the impulse resp of the LPF

fig_step_lpf = plot(t, y_lpf)
xlabel!("t (s)");
ylabel!("y(t)")
title!("Plot of the response y(t) vs time")

display(fig_step_lpf)

From the plot, the maximum value occurs at $y(t) = 1.090927$ and $y(t) = -0.090927$. This implies that the 10% and 90% values are $y(t) = 0.0273$ and $y(t) = 0.9727$, respectively, as calculated in the code below.  

In [ ]:
y_max = 1.090927;
y_min = -0.090927;

y90 = y_max - (y_max - y_min)*0.1;
y10 = y_min + (y_max - y_min)*0.1;

println("The 10% value is y(t_10) = $y10 and the 90% value is y(t_90) = $y90.")

The 10% to 90% rise time corresponds to the length of time on the $x$-axis it takes to go from $y(t_{10})$ and $y(t_{90})$. This corresponds to $t_{10} = -0.118$ and $t_{90} = 0.278$, which yields a time length of $0.40 s$. This value is close to the expected value of $0.44 s$.

# Plotting Periodic functions

The code below generates a plot of $4cos(20\pi t)$ as a function of t from $0$ to $1$ with a time interval of width $\Delta t = 0.001$.

In [ ]:
Δt = 0.001          # time step
t = 0:Δt:1;        # steps of 0.01s from 0s to 1s

A_20 = 4
f20 = 10 
w20 = 2*pi*f20

cos20 = A_20*cos.(w20*t)

fig_cos20 = plot(t, cos20)
xlabel!("t (s)")
ylabel!("cos20")
title!("Plot of sinusoid with amplitude 4 against time")

display(fig_cos20)

# 2.x.4 b)

The code below generates a plot of $2cos(30\pi t)$ as a function of t from $0$ to $1$ with a time interval of width $\Delta t = 0.001$.

In [ ]:
A_30 = 2
f30 = 15 
w30 = 2*pi*f30

cos30 = A_30*cos.(w30*t)

fig_cos30 = plot(t, cos30)
xlabel!("t (s)")
ylabel!("cos30")
title!("Plot of sinosoid with amplitude 2 against time")

display(fig_cos30)

We combine the two functions by addition to produce $\nu (t)$ and generate the plot using the code below.

In [ ]:
v_t = cos20 + cos30

fig_v = plot(t, v_t)
xlabel!("t (s)")
ylabel!("v(t)")
title!("Plot of v(t) vs t")

display(fig_v)

From the plot above, we can measure the period by using the cursor to measure the value of $t$ at the start and end of one cycle. The period is the length of time between these two point.

$t_1 = 0.179 s$

$t_2 = 0.379 s$

$\implies T = t_2 - t_1 = 0.2 s$

$\therefore f_0 = 1/0.2 = 5 Hz$

where $f_0$ is the fundamental frequency in Hz. Alternatively,

$\omega_0 = 2\pi f_0 =  10\pi  rad \cdot s^{-1}$

This result is equal to the expected value as calculated in Drill Problem 2.12.

# Plotting Magnitude and Phase

Suppose that $R = 100 \Omega$ and $C = 100 nF$. We can generate the magnitude spectrum of the frequency response $H(\omega)$ given by

$H(\omega) = \frac{j \omega RC}{1 + j\omega RC}$ 

using the code below. We note that the magnitude $|H(\omega)|$ of the transfer function is given by

$|{H(\omega)}| = \frac{\omega R C}{\sqrt{1 + \omega^2 R^2 C^2}}$.

In [ ]:
Δw = 0.1
w = -10000:Δw:10000

R = 100
C = 100*10^(-9)
Hw = (j*R*C*w)./(1 .+ j*R*C*w);
mag_Hw = abs.(Hw)

fig_magnitude = plot(w, mag_Hw)
xlabel!("Frequency in rad/s")
ylabel!("|H(w)|")
title!("Transfer function of a passive HPF")

display(fig_magnitude)

The phase spectrum is given by

$\arg{(H(\omega))} = \tan^{-1}\left( \frac{1}{\omega RC} \right)$

and the plot is generated below.

In [ ]:
angle_Hw = angle.(Hw)

fig_angle = plot(w, angle_Hw)
xlabel!("Frequency in rad/s")
ylabel!("arg(H(w))")
title!("Phase spectrum of H(w)")


display(fig_angle)

The real part of the transfer function $H(\omega)$ of a first-order passive HPF is expressed as

$Re\{H(\omega)\} =  \frac{\omega^2 R^2 C^2}{1+\omega^2 R^2 C^2}$

and can be represented graphically using the code below.

In [ ]:
re_Hw = real(Hw)

fig_real = plot(re_Hw)
xlabel!("Frequency in rad/s")
ylabel!("Re{H(w)}")
title!("Plot of the real part of H(w)")

display(fig_real)

The real part of the transfer function of the part HPF filter contributes a component that makes the magnitude of the frequency response point upward and symmetric about the y-axis.

The imaginary part of the transfer function is expressed as 

$Im\{H(\omega)\} = \frac{\omega R C}{1 + \omega^2 R^2 C^2}$

and is represent graphically by the code below.

In [ ]:
im_Hw = imag(Hw)

fig_im = plot(w, im_Hw)
xlabel!("Frequency in rad/s")
ylabel!("Im{H(w)}")
title!("Plot of the imaginary part of of the spectrum")

display(fig_im)

We see that the imaginary component contributes linear behaviour to the magnitude of the transfer function.

Using the values $R = 100 \Omega$ and $C = 100nF$, the expected cut-off frequency $\omega_c$ is

$\omega_c = \frac{1}{(100) (100 \times 10^{-9})} = 100 kHz$

We can determine the value of the magnitude of the frequency response at the cut-off frequency (i.e. $|H(100 kHz)|$) graphically or numerically using the below code.

In [ ]:
wc = 100000 # cut-off frequency in Hz
Hc = (wc*R*C)/(sqrt(1+(wc*R*C)))

println("The value of the magnitude of the filter response at the cut-off frequency is equal to $Hc")

The value obtained corresponds to the point where the magnitude of the response is equal to 70.7% of the maximum. We can determine the ratio of between the value of the magnitude when the frequency is infinitly large ($|H(inf)|$) and the value at the cut-off frequency ($|H(\omega_c)|$), i.e.

$\alpha = \frac{|H(\infty)|}{|H(\omega_c)|}$

in decibel (dB) and where,

$|H(\infty)| = \lim_{\omega \rightarrow \infty} |H(\omega)|$

$\implies |H(\infty)| = 1$

In [ ]:
H_inf = 1 

α = H_inf/Hc
α   = 10*log10(α)
println("The ratio |H(inf)| : |H(wc)| is $α dB.")

# Simulating a Noise Waveform

In this section, we implement code to simulate the output of the LPF by adding impulse shifted responses. 

In [ ]:
B = 1
t_max=100 
Δt=0.1
t=0:Δt:t_max; 

N_t = length(t) 
N = 100

Sa(t) = sin(t)/t

A = zeros(N);
τ = zeros(N);
out_put = zeros(N_t); 

for n=1:N
    τ[n] = rand() * t_max
    A[n] = 2*round(rand()) - 1
    out_put .+= A[n]*2*B*Sa.(2*B*(t .- τ[n]) )
end


# Plot the input impulses

Next, we plot the weights A[n] at the corresponding times tau[n].

In [ ]:
fig_impulses = plot(τ, A, line=:stem, marker=:circle, markersize=2);

xlabel!("t (s)")
ylabel!("Dirac deltas")
title!("2.6.1 Plot of Dirac functions to be passed through filter")

display(fig_impulses)

# Plot of the Sa-functions Superimposed

A for loop generates the Sa-function resulting from passing each Dirac function through the LPF.

In [ ]:
fig_safnctns = plot(t, A[1]*2*B*Sa.(2*B*(t .- τ[1])), label="" );

xlabel!("t (s)")
ylabel!("x(t)")
title!("2.6.2 Plot of Superimposed Sa-functions from passing Dirac functions through LPF")

for n=2:N
    plot!(t, A[n]*2*B*Sa.(2*B*(t .- τ[n])), label="" ); 
end

display(fig_safnctns)

# Plot final out of superimposed Sa-functions

The Gaussian white noise is simulated by summing the Sa-functions from passing Dirac deltas through a LPF.

In [ ]:
fig_out = plot(t, out_put);
xlabel!("t (s)")
ylabel!("n(t)")
title!("2.6.3 Gaussian white noise plot simulated by passing Dirac functions through LPF")

display(fig_out)


# Histogram of Output Values

We investigate generated random values to see if the standard deviation of the Gaussian white noise models the normal probability density function.

In [ ]:
fig_histogram = histogram(out_put,nbins=20)
display(fig_histogram)

We investigate how changing the bandwidth $B$ affects the appearance of the waveform $n(t)$ and its resulting histogram. We compare the results for the cases when $B_0 = 0.5$, $B = 1$ and $B_1 = 2$.

In [ ]:
B_0 = 0.5
B_1 = 2
out_put = zeros(N_t) # reset output to zeros
# Dirac impulses when the bandwidth is halved
for n=1:N
    τ[n] = rand() * t_max
    A[n] = 2*round(rand()) - 1
    out_put .+= A[n]*2*B_0*Sa.(2*B_0*(t .- τ[n]) )
end


fig_impulsesB0 = plot(τ, A, line=:stem, marker=:circle, markersize=2);

xlabel!("t (s)")
ylabel!("Dirac deltas")
title!("2.6.4 Plot of Dirac functions to be passed through LPF with B = 0.5 Hz")

display(fig_impulsesB0)


# Plotting Sa-functions for 
# fig_safnctns0 = plot(t, A[1]*2*B_0*Sa.(2*B_0*(t .- τ[1])), label="" );

# xlabel!("t (s)")
# ylabel!("x(t)")
# title!("2.6.5 Superimposed Sa-functions from passing Dirac functions through LPF with B = 0.5")

# for n=2:N
#     plot!(t, A[n]*2*B_0*Sa.(2*B_0*(t .- τ[n])), label="" ); 
# end

# display(fig_safnctns0)

fig_out0 = plot(t, out_put);
xlabel!("t (s)")
ylabel!("n(t)")
title!("2.6.6 Gaussian white noise simulated by passing Dirac functions through LPF with B = 0.5 Hz")

display(fig_out0)

fig_histogram0 = histogram(out_put,nbins=20)
display(fig_histogram0)


In [ ]:
out_put = zeros(N_t)
for n=1:N
    τ[n] = rand() * t_max
    A[n] = 2*round(rand()) - 1
    out_put .+= A[n]*2*B_1*Sa.(2*B_1*(t .- τ[n]) ) # B_1 = 2 Hz
end


fig_impulsesB1 = plot(τ, A, line=:stem, marker=:circle, markersize=2);

xlabel!("t (s)")
ylabel!("Dirac deltas")
title!("2.6.7 Plot of Dirac functions to be passed through LPF with B = 2 Hz")

display(fig_impulsesB1)

# Plotting Sa-functions for 
# fig_safnctns1 = plot(t, A[1]*2*B_1*Sa.(2*B_1*(t .- τ[1])), label="" );

# xlabel!("t (s)")
# ylabel!("x(t)")
# title!("2.6.8 Superimposed Sa-functions from passing Dirac functions through LPF B = 2")

# for n=2:N
#     plot!(t, A[n]*2*B_1*Sa.(2*B_1*(t .- τ[n])), label="" ); 
# end

# display(fig_safnctns1)

fig_out1 = plot(t, out_put);
xlabel!("t (s)")
ylabel!("n(t)")
title!("2.6.9 Gaussian white noise simulated by passing Dirac functions through LPF B = 2 Hz")

display(fig_out1)

fig_histogram1 = histogram(out_put,nbins=20)
display(fig_histogram1)

The results show that increasing the bandwidth in the time domain decreases the spacing between the noise peaks representing superimposed sa-functions. When the bandwidth is $B = 0.5$, the noise peaks are wider and more spread out compared to the peaks in the case where $B = 1$ or $B = 2$. When $B = 2$, the peaks are closer together. The cause of this effect is seen clearly in the plots of the sa-functions. The larger the bandwidth $B$, the more sa-functions are seen within a smaller interval along the horizontal axis. We also know that the width of the main lobe of a Sa-function is given by

$\delta t  \approx 0.44/B $

Therefore, decreasing $B$ increases the with of the main lobes of the sa-function. The overall effect results in wider peaks for decreased bandwidth $B$.
The height of the peaks increases with an increase in frequency. When $B = 0.5$, the heights of the peaks range from -4 to 4; when $B = 1$, the value of the peaks are in the interval [-6, 8] and when $B = 2$, the values are in the range [-10, 10].

The widening of the maximum values corresponds to the widening of the base of the histogram. In other words, the standard deviation $\sigma$ between the generated values increases as the bandwidth is increased. The Gaussian noise holds values between $-\sigma$ and $+\sigma$ for 68% percent of the time. As the bandwidth frequency is increased, the values which the noise holds for 68% of the time also increase.

# Making the weights any value in the range [-1, 1]

The following generates Dirac functions with weights in the range [-1, 1]. 

In [ ]:
out_put = zeros(N_t)
for n=1:N
    τ[n] = rand() * t_max
    A[n] = 2*rand() - 1
    out_put .+= A[n]*2*B*Sa.(2*B*(t .- τ[n]) )
end

fig_impulsesA = plot(τ, A, line=:stem, marker=:circle, markersize=2);

xlabel!("t (s)")
ylabel!("Dirac deltas")
title!("2.6.10 Plot of Dirac functions to be passed through LPF")

display(fig_impulsesA)

# Plotting Sa-functions for 
# fig_safnctnsA = plot(t, A[1]*2*B*Sa.(2*B*(t .- τ[1])), label="" );

# xlabel!("t (s)")
# ylabel!("x(t)")
# title!("2.6.11 Superimposed Sa-functions from passing Dirac functions through LPF")

for n=2:N
    plot!(t, A[n]*2*B*Sa.(2*B*(t .- τ[n])), label="" ); 
end

# display(fig_safnctnsA)

fig_outA = plot(t, out_put);
xlabel!("t (s)")
ylabel!("n(t)")
title!("2.6.12 Gaussian white noise simulated by passing Dirac functions through LPF")

display(fig_outA)

fig_histogramA = histogram(out_put,nbins=20)
display(fig_histogramA)

We see that changing the weights of the Dirac deltas such that they can take any value between [-1, 1] produces Dirac functions of different height, which results in the possible values of the height of the sa-functions obtained from passing the delta functions through a LPF. We see from the histogram that the values still behave uniformly. The standard deviation between the generated values remains approximately the same.

We increase the number of samples by widening the time domain and increasing the of Dirac impulses.

In [ ]:
t_max=1000 
Δt=0.1
t=0:Δt:t_max; 

N_t = length(t) 
N = 1000

Sa(t) = sin(t)/t

A = zeros(N);
τ = zeros(N);
out_put = zeros(N_t); 

for n=1:N
    τ[n] = rand() * t_max
    A[n] = 2*round(rand()) - 1
    out_put .+= A[n]*2*B*Sa.(2*B*(t .- τ[n]) )      # using B = 1 Hz
end

fig_histogrammax = histogram(out_put,nbins=20)
display(fig_histogrammax)

The shape of the histogram more accurately approximates the Gaussian distribution given by the probability density function. 